# Langchain Components

1. **PromptTemplate**: Allows reusable templates.
2. **Nodes**: Agents or functions
3. **Edges**: connect nodes
4. **Connectional Edges**: decisions
5. **Agent State**: 
    - Agent State is accessible to all parts of the graph
    - It is local to the graph
    - Can be stored in a persistence layer


Simple Agent State

In [25]:
# class AgentState(TypedDict):
#         messages: Annotated[Sequence[BaseMessage], operator.add]

Complex Agent State

In [26]:
# class AgentState(TypedDict):
#     input: str
#     chat_history: list[BaseMeassage]
#     agent_outcome: Union[AgentAction, AgentFinish, Name]
#     intermediate_step: Annotated[list[tuple[AgentAction, str]], operated.add]

**Code High Level Overview**

1. llm:call_ll
2. condn_edge:exists_action
3. action:take_action

In [27]:
from langgraph.graph import StateGraph, END

# to construct agent state
from typing import TypedDict, Annotated
import operator

# to construct human, ai and system messages
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage

# llm 
from langchain_ollama import ChatOllama

# tool
from langchain_community.tools.tavily_search import TavilySearchResults

Tool

In [28]:
import os
TAVILY_API_KEY = "tvly-IsSIq4iFinMqzSVRyEa..."
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [40]:
tool = TavilySearchResults(max_results=2)
tool.invoke({"query": "What happened in the latest burning man floods"})

[{'url': 'https://www.nbcnews.com/news/us-news/live-blog/live-updates-burning-man-flooding-keeps-thousands-stranded-nevada-site-rcna103193',
  'content': "Profile\nSections\ntv\nFeatured\nMore From NBC\nFollow NBC News\nnews Alerts\nThere are no new alerts at this time\nBurning Man flooding keeps thousands stranded at Nevada site as authorities investigate 1 death\nBurning Man attendees struggling to get home\n70,000+ stuck at Burning Man: When will they be able to get out?\n Thousands still stranded at Burning Man after torrential rain\nBurning Man revelers unfazed by deluge and deep mud\nReuters\nThousands of Burning Man attendees partied hard on Sunday despite downpours that turned the Nevada desert where the annual arts and music festival takes place into a sea of sticky mud and led officials to order the multitudes to shelter in place.\n Neal Katyal warns hiking in the mud\ncan be 'worse than walking on ice'\nDoha Madani\nNeal Katyal, the former acting U.S. solicitor general, is a

In [41]:
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


# Agent State

In [42]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

# Agent Class

In [44]:
class Agent:
    def __init__(self, model, tools, system=""):
        self.system = system
        # Now let us initialize Graph
        # 1st is state graph with Agent State [it does not have any nodes or edges attached to it.]
        graph = StateGraph(AgentState)
        # Sketch out nodes
        graph.add_node("llm",...)
        graph.add_node("action",...)
        # Now conditional edge. this goes after llm is called, 
        # it checks whether there is action present. 
        # If there is, it goes to action node.
        #  And if not it goes to the end node
        graph.add_conditional_edges(
            "llm", # node where the edge starts
            ..., # function that will determine where to go after
            {True: "action", False:END} # dict: representing how to map the response of the fn to the next node to go to. So if the fn returns true, we'll go to action node, else end node.
        )
        graph.add_node("action", "llm") # regular node
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

        def call_openai(self, state: AgentState):
            messages = state['messages']
            if self.system:
                messages = [SystemMessage(content=self.system)] + messages
            message = self.model.invoke(messages)
            return {'messages': [message]}
        
        def take_action(self, state: AgentState):
            tool_calls = state['messages'][-1].tool_calls
            results = []
            for t in tool_calls:
                print(f"Calling: {t}")
                if not t['name'] in self.tools:      # check for bad tool name from LLM
                    print("\n ....bad tool name....")
                    result = "bad tool name, retry"  # instruct LLM to retry if bad
                else:
                    result = self.tools[t['name']].invoke(t['args'])
                results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
            print("Back to the model!")
            return {'messages': results}


# Redefined the above Agent with Actions

In [45]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)##### added
        graph.add_node("action", self.take_action)###### added
        graph.add_conditional_edges(
            "llm",
            self.exists_action,######### added
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [46]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOllama(model="llama3.1")
abot = Agent(model, [tool], system=prompt)

In [47]:
# !pip install pygraphviz

In [48]:
# from IPython.display import Image

# Image(abot.graph.get_graph().draw_png())

In [49]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})
result['messages'][-1].content

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': 'ead36170-4423-4b6f-9dcf-950aa5af22f8', 'type': 'tool_call'}
Back to the model!


'The weather in San Francisco, California currently is sunny with a temperature of 58.5°F (14.7°C) and humidity of 82%. There are no precipitation or clouds expected.'

In [ ]:
result['messages'][-1].content